In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision
from torchvision import transforms
import os
import cv2

In [ ]:
base_dir=r'/content/drive/MyDrive/PPG/PPG'

In [ ]:
train_dir = os.path.join(base_dir , 'train')#Test data set directory
test_dir = os.path.join(base_dir , 'test') #Directory of data sets to detect

In [ ]:
transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

In [ ]:
train_ds =  torchvision.datasets.ImageFolder(
        train_dir,
        transform=transform
    )#Test data set
test_ds =  torchvision.datasets.ImageFolder(
        test_dir,
        transform=transform
    )#Detect the dataset and convert the image format into a dataset

In [ ]:
BTACH_SIZE = 32#Adjustable parameters,video memory is insufficient when the parameters must be adjusted
train_dl = torch.utils.data.DataLoader(
                      train_ds,
                      batch_size=BTACH_SIZE,
                      shuffle=True
)
test_dl = torch.utils.data.DataLoader(
                      test_ds,
                      batch_size=BTACH_SIZE,
)
#Select resnet18 for training
model = torchvision.models.resnet18(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False#All the above parameters remain unchanged
in_f = model.fc.in_features#Make sure the last parameter is passed in exactly
model.fc = nn.Linear(in_f, 6)#Replace the last fc layer,divided into four types
if torch.cuda.is_available():
    model.to('cuda')
loss_fn = nn.CrossEntropyLoss()
#Choose adam optimizer,choosethis is relatively high accuracy
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)

In [ ]:
#training
def fit(epoch, model, trainloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in trainloader:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    test_correct = 0
    test_total = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total
    
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc
 
epochs = 10
 
train_loss = []
train_acc = []
test_loss = []
test_acc = []
 
for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,
                                     model,
                                     train_dl,
                                     test_dl)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

epoch:  0 loss：  0.056 accuracy: 0.232 test_loss：  0.066 test_accuracy: 0.269
epoch:  1 loss：  0.056 accuracy: 0.26 test_loss：  0.066 test_accuracy: 0.269
epoch:  2 loss：  0.056 accuracy: 0.232 test_loss：  0.066 test_accuracy: 0.269
epoch:  3 loss：  0.056 accuracy: 0.276 test_loss：  0.066 test_accuracy: 0.278
epoch:  4 loss：  0.056 accuracy: 0.276 test_loss：  0.066 test_accuracy: 0.269
epoch:  5 loss：  0.055 accuracy: 0.276 test_loss：  0.066 test_accuracy: 0.259
epoch:  6 loss：  0.056 accuracy: 0.236 test_loss：  0.066 test_accuracy: 0.269
epoch:  7 loss：  0.055 accuracy: 0.309 test_loss：  0.066 test_accuracy: 0.259
epoch:  8 loss：  0.055 accuracy: 0.289 test_loss：  0.066 test_accuracy: 0.259
epoch:  9 loss：  0.055 accuracy: 0.289 test_loss：  0.066 test_accuracy: 0.269
epoch:  10 loss：  0.055 accuracy: 0.305 test_loss：  0.066 test_accuracy: 0.269
epoch:  11 loss：  0.055 accuracy: 0.297 test_loss：  0.066 test_accuracy: 0.269
epoch:  12 loss：  0.055 accuracy: 0.309 test_loss：  0.066 test_